# Manual ingest workflow, bypassing GUI

## Login

Either log in via a local config file (see [01_pipeline](./01_pipeline.ipynb)), or enter login information manually. If you are don't have your login information, contact the administrator.


In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, session, equipment, surgery, event, trial, imaging, behavior, model
from adamacs.ingest import session as isess
from adamacs.helpers import stack_helpers as sh
from adamacs.ingest import behavior as ibe
import pathlib
from natsort import natsorted, ns
import datajoint as dj
from rspace_client.eln import eln
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dj.__version__

[2023-11-07 15:09:02,493][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-11-07 15:09:02,540][INFO]: Connected tobiasr@172.26.128.53:3306


'0.14.1'

### RSpace connection

In [2]:
URL=dj.config['custom'].get('rspace_URL')
API_KEY=dj.config['custom'].get('rspace_API_key')
api = eln.ELNClient(URL, API_KEY)
api.get_status()

{'message': 'OK', 'rspaceVersion': '1.89.2'}

## Activation
Next, import from `adamacs.pipeline` to activate the relevant schema.

In [3]:
from adamacs.utility import *
# from adamacs.nbgui import *
from adamacs.pipeline import subject, session, surgery, scan, equipment

Assign easy names for relevant tables

In [4]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath()
    )

some helpers

In [5]:

def get_session_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]
     
def get_scan_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]

def get_session_key_from_dir(string):
    result = [re.search(r'sess\S+', item).group(0) for item in string]
    return result

def get_user_initials_from_dir(string):
    result = [name[:2] for name in string]
    return result

def get_subject_key_from_dir(string):
    result = [item.split("_")[1] for item in string]
    return result

def get_date_key_from_dir(directory):
    return directory.split("_")[-1]

def get_scan_key_from_dir(string):
    result = [re.search(r'scan\S+_', item).group(0)[:-1] for item in string]
    return result

## Parse directories

In [6]:
# get content of user directory
import fnmatch
dataroot = dj.config['custom']['exp_root_data_dir'][0]
# dirs_root = [d for d in os.listdir(dataroot) if os.path.isdir(os.path.join(dataroot, d)) and '_' in d]
dirs_root = [d for d in os.listdir(dataroot) if os.path.isdir(os.path.join(dataroot, d)) and fnmatch.fnmatch(d, 'JJ*') and fnmatch.fnmatch(d, '*1-*')]
sorted_dirs_root = natsorted(dirs_root, key=get_date_key_from_dir, reverse = True)
sorted_dirs_root

['JJ_ROS-1604_2023-11-02_scan9FKW82R3_sess9FKW82R3',
 'JJ_ROS_1438_2022-11-22_scan9FF6TL96_sess9FF6TL96']

In [7]:
sessi = "sess9FKW82R3"
scansi = "scan9FKW82R3"

## Session ingest

In [8]:
Project = project.fetch('project')
Equipment = equipment.Equipment().fetch('scanner')
Recording_Location = surgery.AnatomicalLocation().fetch('anatomical_location')
s2pparm = imaging.ProcessingParamSet.fetch("paramset_idx", "paramset_desc")
DLCModels = model.Model.fetch("model_name")

print(project)
print(Equipment)
print(Recording_Location)
print(s2pparm)
print(DLCModels)

*project       project_descri
+------------+ +------------+
ATN            ATN-functional
dummy          dummy         
hpc-repstab    hpc-representa
rsc-functop    rsc-functional
rsc-hpc        rsc-hippocampa
rsc-latent     rsc-contextual
sc-lgn-actvis  sc-lgn-active-
V1-oddball     v1-oddball-pre
vc-lgn-repstab vc-lgn-represe
 (Total: 9)

['bench2p' 'dummy' 'macroscope' 'mini2p_01' 'mini2p_02' 'mini2p_03'
 'mini2p_04' 'mini2p_05']
['ATN' 'Ctx' 'dCA1' 'DG' 'dummy' 'LGNV1' 'RSCa' 'RSCg' 'V1']
[array([0, 1, 2, 3, 4, 5, 6]), array(['TR: Mini2p (new, non-rigid, individual scans, reg_tiff for movie, custom classifier)',
       'TR: Mini2p (rigid, mini2p classifier, no concat, REGTIFF)',
       'TR: Trondheim Mini2p (non-rigid, built-in classifier, scan_concat)',
       'TR: Bench2p (non-rigid, custom classifier, reg_tiff for movie, scans individually)',
       'TR: Bench2p (rigid, custom bench2p classifier, individual, SAVETIF)',
       'TR: Mini2p (rigid, built-in classifier, individual)'

Ingest Session and Scan

In [9]:
isess.ingest_session_scan(sessi, verbose=True, project_key="rsc-functop", equipment_key="mini2p_01", location_key="RSCa", software_key='ScanImage')
session.SessionSameSite.update1({'session_id': sessi, 'same_site_id': sessi})


Number of scans found: 1
[['scan9FKW82R3']]
['JJ_ROS-1604_2023-11-02_scan9FKW82R3_sess9FKW82R3']


/tmp/ipykernel_3173673/3791184197.py:1: UserWarning: 
Skipped existing session row: ('sess9FKW82R3', <module 'adamacs.schemas.subject' from '/home/tobiasr/adamacs/adamacs/schemas/subject.py'>, '2023-11-02')
  isess.ingest_session_scan(sessi, verbose=True, project_key="rsc-functop", equipment_key="mini2p_01", location_key="RSCa", software_key='ScanImage')
/tmp/ipykernel_3173673/3791184197.py:1: UserWarning: 
Skipped existing ProjectSession row: ('rsc-functop', 'sess9FKW82R3')
  isess.ingest_session_scan(sessi, verbose=True, project_key="rsc-functop", equipment_key="mini2p_01", location_key="RSCa", software_key='ScanImage')
/tmp/ipykernel_3173673/3791184197.py:1: UserWarning: 
Skipped existing SessionDirectory: ('sess9FKW82R3', PosixPath('/datajoint-data/data/tobiasr/JJ_ROS-1604_2023-11-02_scan9FKW82R3_sess9FKW82R3'))
  isess.ingest_session_scan(sessi, verbose=True, project_key="rsc-functop", equipment_key="mini2p_01", location_key="RSCa", software_key='ScanImage')
/tmp/ipykernel_3173673

In [10]:
scan.Scan * session.SessionSameSite * session.Session() & f'session_id = "{sessi}"'

session_id,scan_id,scanner,acq_software,scan_notes free-notes,same_site_id,subject PyRat import uses this for earmark value,session_datetime
sess9FKW82R3,scan9FKW82R3,mini2p_01,ScanImage,,sess9FKW82R3,ROS-1604,2023-11-02 00:00:00


In [11]:
query = scan.ScanPath() & 'scan_id = "' + scansi + '"'
dir_proc = query.fetch('path')[0]
print(dir_proc)

/datajoint-data/data/tobiasr/JJ_ROS-1604_2023-11-02_scan9FKW82R3_sess9FKW82R3


In [12]:
# push scan to ProcessingTask
# TODO: handle multiscan concatenation from here?
selected_s2pparms_index = 0
imaging.ProcessingTask.insert1((sessi, scansi, selected_s2pparms_index, dir_proc, 'trigger'), skip_duplicates=True)

In [ ]:
# imaging.ProcessingTask.delete()

In [ ]:
# POPULATE!
populate_settings = {'display_progress': True, 'suppress_errors': False, 'processes': 1}
scan.ScanInfo.populate(**populate_settings) 

## Ingest AUX

In [ ]:
aux_setup_typestr = (scan.ScanInfo() & 'scan_id = "' + scansi + '"').fetch("userfunction_info")[0]    
print(aux_setup_typestr)

In [ ]:
ibe.ingest_aux(sessi,scansi,verbose=True, aux_setup_type=aux_setup_typestr)

## Run processing jobs


In [ ]:
imaging.Processing.populate(**populate_settings)

### Curation

In [ ]:
imaging.Curation().create1_from_processing_task({'session_id': sessi, 'scan_id': scansi, "paramset_idx": selected_s2pparms_index, "manual_curation": 0})

imaging.MotionCorrection.populate(**populate_settings)

imaging.Segmentation.populate(**populate_settings)

imaging.MaskClassification.populate(**populate_settings)

imaging.Fluorescence.populate(**populate_settings)

imaging.Activity.populate(**populate_settings)

## Pose estimation

In [ ]:
aux_setup_typestr = (scan.ScanInfo() & 'scan_id = "' + scansi + '"').fetch("userfunction_info")[0]
selected_DLCmodel = 'Head_orientation-NK'
print('- - - -')
print('DLC pose estimation:', scansi)

# insert TOP movie into model table
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0] 
moviepath = str(list(pathlib.Path((scan.ScanPath() & scan_key).fetch("path")[0]).glob("*top*.mp4*"))[0])

key = {'session_id': scan_key["session_id"],
    'recording_id': scan_key["scan_id"], 
    'camera': "mini2p1_top", # Currently 'scanner' due to in equipment tables
    }
model.VideoRecording.insert1(key, skip_duplicates=True)

key.update({'file_path': moviepath,
            'file_id': 0})  #INCREMENT FILE_ID WITH CAM NUMBER?

model.VideoRecording.File.insert1(key, ignore_extra_fields=True, skip_duplicates=True)

In [ ]:
model.VideoRecording.File * model.VideoRecording() &  'recording_id = "' + scansi + '"'

In [ ]:
model.Model.drop()

In [ ]:
key =  (model.VideoRecording & f'recording_id="{scansi}"').fetch1('KEY')
key.update({'model_name': selected_DLCmodel, 'task_mode': 'trigger'}) 
key      


In [ ]:
# run pose estimation

model.PoseEstimationTask.insert_estimation_task(key, key["model_name"], analyze_videos_params={'save_as_csv':True, 'dynamic':(True,.5,60)}) # dynamic cropping

## Cleaning. Use with caution!

In [ ]:
# subject.Subject.delete()
# session.Session.delete()
# imaging.Processing.delete()
# imaging.Curation.delete()
# event.Event.delete()
# event.BehaviorRecording.delete()

In [ ]:
# (session.Session & "session_id LIKE 'sess9FKW82R3%'").delete()
(imaging.ProcessingTask & "session_id LIKE 'sess9FKNBQCS%'").delete()
# (subject.Subject & "subject = 'ROS-1571'").delete()
# # subject.Subject.delete()
# # session.Session.delete()
# # imaging.Processing.delete()
# # imaging.Curation.delete()
# # event.Event.delete()
# # event.BehaviorRecording.delete()

In [ ]:
# session.Session.drop()
# scan.Scan.drop()
# imaging.Processing.drop()
# imaging.Curation.drop()